In [1]:
path_repo_benchmark = "./data/corpus_benchmark"

### renommer 

In [3]:
from pathlib import Path

path_repo_benchmark = Path("./data/corpus_benchmark")

for p in path_repo_benchmark.glob("Copy of FRA*.jpg"):
    new_name = p.name.replace("Copy of ", "", 1)
    target = p.with_name(new_name)
    p.rename(target)

print("done")


done


### créer jsonl de validation

In [4]:
import json
from pathlib import Path

path_repo_benchmark = Path("./data/corpus_benchmark")
out_path = Path("data/corpus_benchmark.jsonl")

items = []
for p in sorted(path_repo_benchmark.glob("FRA*.jpg")):
    items.append({
        "nom_image": p.name,
        "raw_text": "",
        "city": "",
        "hamlet": "",
        "monument": "",
        "other": ""
    })

with out_path.open("w", encoding="utf-8") as f:
    for obj in items:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

print("ok ->", out_path, "count:", len(items))


ok -> data/corpus_benchmark.jsonl count: 50


# corpus test

In [5]:
from pathlib import Path

path = Path("data/test20/corpus_test20")

for p in path.iterdir():
    if p.is_file() and p.name.startswith("copy_"):
        new_name = p.name.replace("copy_", "", 1)
        p.rename(p.with_name(new_name))

print("done")


done


In [14]:
import json
from pathlib import Path

path_meta = Path("data/metadata.json")
path_trans = Path("data/test20/transcriptions_coherentes_final.json")
out_path = Path("data/test20/metadata_test20.json")

meta = json.loads(path_meta.read_text(encoding="utf-8"))
trans = json.loads(path_trans.read_text(encoding="utf-8"))

wanted = {t.get("file_name") for t in trans if t.get("file_name")}

file_names = meta.get("File name", [])
keep_idx = [i for i, name in enumerate(file_names) if name in wanted]

filtered = {}
for key, values in meta.items():
    if isinstance(values, list):
        filtered[key] = [values[i] for i in keep_idx]
    else:
        filtered[key] = values

out_path.write_text(json.dumps(filtered, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", out_path, "rows:", len(keep_idx))


ok -> data/test20/metadata_test20.json rows: 20


In [15]:
import json
from pathlib import Path

path_meta = Path("data/test20/metadata_test20.json")
path_trans = Path("data/test20/transcriptions_coherentes_final.json")
out_path = Path("data/test20/metadata_test20_enrichi.json")

meta = json.loads(path_meta.read_text(encoding="utf-8"))
trans = json.loads(path_trans.read_text(encoding="utf-8"))

# Index par nom de fichier
trans_by_name = {t.get("file_name"): t for t in trans}

file_names = meta.get("File name", [])
n = len(file_names)

# toutes les clés présentes dans transcriptions (sauf file_name)
trans_keys = set()
for t in trans:
    for k in t.keys():
        if k != "file_name":
            trans_keys.add(k)

# colonnes cibles (prefixées pour éviter collision)
col_map = {k: f"trans_{k}" for k in sorted(trans_keys)}

# initialise les colonnes
for col in col_map.values():
    if col not in meta:
        meta[col] = [""] * n

# remplissage
for i, fname in enumerate(file_names):
    t = trans_by_name.get(fname)
    if not t:
        continue
    for src_key, dst_key in col_map.items():
        meta[dst_key][i] = t.get(src_key, "")

out_path.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", out_path)


ok -> data/test20/metadata_test20_enrichi.json


In [16]:
import json
import csv
from pathlib import Path

in_path = Path("data/test20/metadata_test20_enrichi.json")
out_path = Path("data/test20/metadata_test20_enrichi.csv")

data = json.loads(in_path.read_text(encoding="utf-8"))

fields = list(data.keys())
n = len(data.get("File name", []))

with out_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    for i in range(n):
        row = []
        for k in fields:
            v = data.get(k, "")
            if isinstance(v, list):
                row.append(v[i] if i < len(v) else "")
            else:
                row.append(v)
        writer.writerow(row)

print("ok ->", out_path)


ok -> data/test20/metadata_test20_enrichi.csv


In [17]:
import json
from pathlib import Path

in_path = Path("data/metadata_monu.json")
out_path = Path("data/metadata_monu_52.json")

data = json.loads(in_path.read_text(encoding="utf-8"))

# longueur basée sur File name
n = min(52, len(data.get("File name", [])))

# supprimer colonnes
data.pop("longitude", None)
data.pop("latitude", None)

# tronquer toutes les colonnes listes
for k, v in data.items():
    if isinstance(v, list):
        data[k] = v[:n]

out_path.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", out_path)


ok -> data/metadata_monu_52.json


In [18]:
import json
import csv
from pathlib import Path

in_path = Path("data/metadata_monu_52.json")
out_path = Path("data/metadata_monu_52.csv")

data = json.loads(in_path.read_text(encoding="utf-8"))

fields = list(data.keys())
n = len(data.get("File name", []))

with out_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    for i in range(n):
        row = []
        for k in fields:
            v = data.get(k, "")
            if isinstance(v, list):
                row.append(v[i] if i < len(v) else "")
            else:
                row.append(v)
        writer.writerow(row)

print("ok ->", out_path)


ok -> data/metadata_monu_52.csv


In [19]:
import json
import csv
from pathlib import Path

path_json = Path("data/metadata.json")
path_csv = Path("data/metadata.csv")

remove_cols = {
    "City", "Raw text", "Hamlet", "Monument", "Other", "coordonnées"
}
rename_from = "code_insee"
rename_to = "code_insee_commune"

# --- JSON ---
data = json.loads(path_json.read_text(encoding="utf-8"))

# remove columns
for col in remove_cols:
    data.pop(col, None)

# rename column
if rename_from in data:
    data[rename_to] = data.pop(rename_from)

path_json.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")

# --- CSV ---
with path_csv.open("r", encoding="utf-8") as f_in:
    reader = csv.DictReader(f_in)
    fieldnames = []
    for fn in reader.fieldnames:
        if fn in remove_cols:
            continue
        if fn == rename_from:
            fieldnames.append(rename_to)
        else:
            fieldnames.append(fn)

    rows = []
    for row in reader:
        new_row = {}
        for k, v in row.items():
            if k in remove_cols:
                continue
            if k == rename_from:
                new_row[rename_to] = v
            else:
                new_row[k] = v
        rows.append(new_row)

with path_csv.open("w", newline="", encoding="utf-8") as f_out:
    writer = csv.DictWriter(f_out, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(rows)

print("ok ->", path_json, "and", path_csv)


ok -> data/metadata.json and data/metadata.csv


In [21]:
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [22]:
import torch

ModuleNotFoundError: No module named 'torch'

In [23]:
!pip install torch

  Using cached torch-2.9.1-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached torch-2.9.1-cp313-none-macosx_11_0_arm64.whl (74.5 MB)
Using cached fsspec-2025.12.0-py3-none-any.whl (201 kB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [sympy]^C


In [25]:
path = "./vaucluse/truc.txt"

In [26]:
with open(path, "r", encoding="utf-8") as f:
    for line in f:
        print(line)

FileNotFoundError: [Errno 2] No such file or directory: './vaucluse/truc.txt'

In [ ]:
path_metadata = "./data/metadata.json"

In [27]:
import json
from pathlib import Path

path_meta = Path("data/metadata.json")
out_path = Path("data/metadata_output.json")

# Liste des 84 cartes
wanted = {
    "FRAD084_07FI137_0112.jpg",
    "FRAD084_07FI129_0002.jpg",
    "FRAD084_07FI139_0028.jpg",
    "FRAD084_07FI050_0024_01.jpg",
    "FRAD084_07FI087_0275_01.jpg",
    "FRAD084_07FI007_0382.jpg",
    "FRAD084_07FI139_0088.jpg",
    "FRAD084_07FI007_0677.jpg",
    "FRAD084_07FI007_0504.jpg",
    "FRAD084_07FI007_0470.jpg",
    "FRAD084_07FI058_0002_01.jpg",
    "FRAD084_07FI152_0018.jpg",
    "FRAD084_07FI007_0827.jpg",
    "FRAD084_07FI089_0038.jpg",
    "FRAD084_07FI124_0001_01.jpg",
    "FRAD084_07FI087_0114.jpg",
    "FRAD084_07FI007_0699.jpg",
    "FRAD084_07FI007_0513.jpg",
    "FRAD084_07FI097_0001_01.jpg",
    "FRAD084_07FI152_0002.jpg",
    "FRAD084_07FI114_0001_01.jpg",
    "FRAD084_07FI152_0046.jpg",
    "FRAD084_07FI056_0001_01.jpg",
    "FRAD084_07FI087_0185.jpg",
    "FRAD084_07FI007_0719.jpg",
    "FRAD084_07FI087_0310.jpg",
    "FRAD084_07FI069_0005_01.jpg",
    "FRAD084_07FI007_0453.jpg",
    "FRAD084_07FI007_0940.jpg",
    "FRAD084_07FI138_0004.jpg",
    "FRAD084_07FI087_0223_01.jpg",
    "FRAD084_07FI020_0006_01.jpg",
    "FRAD084_07FI007_0586.jpg",
    "FRAD084_07FI087_0267_01.jpg",
    "FRAD084_07FI074_0004.jpg",
    "FRAD084_07FI087_0186_01.jpg",
    "FRAD084_07FI007_0498.jpg",
    "FRAD084_07FI129_0001.jpg",
    "FRAD084_07FI007_0067.jpg",
    "FRAD084_07FI139_0094.jpg",
    "FRAD084_07FI031_0119.jpg",
    "FRAD084_07FI087_0081_01.jpg",
    "FRAD084_07FI072_0001.jpg",
    "FRAD084_07FI152_0004.jpg",
    "FRAD084_07FI139_0035.jpg",
    "FRAD084_07FI003_0027.jpg",
    "FRAD084_07FI007_0915.jpg",
    "FRAD084_07FI152_0071.jpg",
    "FRAD084_07FI007_0834.jpg",
    "FRAD084_07FI007_0714_01.jpg",
    "FRAD084_07FI007_0590.jpg",
    "FRAD084_07FI087_0251_01.jpg",
    "FRAD084_07FI007_1063_01.jpg",
    "FRAD084_07FI124_0003.jpg",
    "FRAD084_07FI007_0158.jpg",
    "FRAD084_07FI089_0029_01.jpg",
    "FRAD084_07FI132_0022_01.jpg",
    "FRAD084_07FI007_0819.jpg",
    "FRAD084_07FI087_0056.jpg",
    "FRAD084_07FI007_0375_01.jpg",
    "FRAD084_07FI042_0002.jpg",
    "FRAD084_07FI139_0059_01.jpg",
    "FRAD084_07FI152_0094.jpg",
    "FRAD084_07FI041_0033.jpg",
    "FRAD084_07FI007_0492_01.jpg",
    "FRAD084_07FI007_0670.jpg",
    "FRAD084_07FI050_0033.jpg",
    "FRAD084_07FI007_0965.jpg",
    "FRAD084_07FI031_0079_01.jpg",
    "FRAD084_07FI087_0298.jpg",
    "FRAD084_07FI139_0152.jpg",
    "FRAD084_07FI087_0300_01.jpg",
    "FRAD084_07FI007_0954.jpg",
    "FRAD084_07FI007_0950.jpg",
    "FRAD084_07FI019_0008_01.jpg",
    "FRAD084_07FI137_0030_01.jpg",
    "FRAD084_07FI087_0122.jpg",
    "FRAD084_07FI003_0073.jpg",
    "FRAD084_07FI152_0020.jpg",
    "FRAD084_07FI123_0042.jpg",
    "FRAD084_07FI081_0007_01.jpg",
    "FRAD084_07FI007_0414.jpg",
    "FRAD084_07FI017_0008.jpg",
    "FRAD084_07FI088_0023_01.jpg",
}

meta = json.loads(path_meta.read_text(encoding="utf-8"))

file_names = meta.get("File name", [])
keep_idx = [i for i, name in enumerate(file_names) if name in wanted]

filtered = {}
for key, values in meta.items():
    if isinstance(values, list):
        filtered[key] = [values[i] for i in keep_idx]
    else:
        filtered[key] = values

out_path.write_text(json.dumps(filtered, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", out_path, "rows:", len(keep_idx))


ok -> data/metadata_output.json rows: 84


In [28]:
import json
from pathlib import Path

path_meta = Path("data/metadata_output.json")
path_trans = Path("data/out_vaucluse/transcriptions_coherentes_final.json")
out_path = Path("data/metadata_output.json")  # overwrite

meta = json.loads(path_meta.read_text(encoding="utf-8"))
trans = json.loads(path_trans.read_text(encoding="utf-8"))

# index transcriptions par nom de fichier
trans_by_name = {t.get("file_name"): t for t in trans}

file_names = meta.get("File name", [])
n = len(file_names)

# toutes les clés de transcriptions (sauf file_name)
trans_keys = set()
for t in trans:
    for k in t.keys():
        if k != "file_name":
            trans_keys.add(k)

# colonnes cibles (préfixées)
col_map = {k: f"trans_{k}" for k in sorted(trans_keys)}

# initialise colonnes si besoin
for col in col_map.values():
    if col not in meta:
        meta[col] = [""] * n

# remplissage
for i, fname in enumerate(file_names):
    t = trans_by_name.get(fname)
    if not t:
        continue
    for src_key, dst_key in col_map.items():
        meta[dst_key][i] = t.get(src_key, "")

out_path.write_text(json.dumps(meta, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", out_path)


ok -> data/metadata_output.json


In [30]:
import json
import csv
from pathlib import Path

in_path = Path("data/metadata_output.json")
out_path = Path("data/metadata_output.csv")

data = json.loads(in_path.read_text(encoding="utf-8"))

fields = list(data.keys())
n = len(data.get("File name", []))

with out_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    for i in range(n):
        row = []
        for k in fields:
            v = data.get(k, "")
            if isinstance(v, list):
                row.append(v[i] if i < len(v) else "")
            else:
                row.append(v)
        writer.writerow(row)

print("ok ->", out_path)


ok -> data/metadata_output.csv


In [32]:
from pathlib import Path
import shutil

src_dir = Path("data/photos")
dst_dir = Path("data/output_vaucluse/images")
dst_dir.mkdir(parents=True, exist_ok=True)

files = [
    "FRAD084_07FI137_0112.jpg",
    "FRAD084_07FI129_0002.jpg",
    "FRAD084_07FI139_0028.jpg",
    "FRAD084_07FI050_0024_01.jpg",
    "FRAD084_07FI087_0275_01.jpg",
    "FRAD084_07FI007_0382.jpg",
    "FRAD084_07FI139_0088.jpg",
    "FRAD084_07FI007_0677.jpg",
    "FRAD084_07FI007_0504.jpg",
    "FRAD084_07FI007_0470.jpg",
    "FRAD084_07FI058_0002_01.jpg",
    "FRAD084_07FI152_0018.jpg",
    "FRAD084_07FI007_0827.jpg",
    "FRAD084_07FI089_0038.jpg",
    "FRAD084_07FI124_0001_01.jpg",
    "FRAD084_07FI087_0114.jpg",
    "FRAD084_07FI007_0699.jpg",
    "FRAD084_07FI007_0513.jpg",
    "FRAD084_07FI097_0001_01.jpg",
    "FRAD084_07FI152_0002.jpg",
    "FRAD084_07FI114_0001_01.jpg",
    "FRAD084_07FI152_0046.jpg",
    "FRAD084_07FI056_0001_01.jpg",
    "FRAD084_07FI087_0185.jpg",
    "FRAD084_07FI007_0719.jpg",
    "FRAD084_07FI087_0310.jpg",
    "FRAD084_07FI069_0005_01.jpg",
    "FRAD084_07FI007_0453.jpg",
    "FRAD084_07FI007_0940.jpg",
    "FRAD084_07FI138_0004.jpg",
    "FRAD084_07FI087_0223_01.jpg",
    "FRAD084_07FI020_0006_01.jpg",
    "FRAD084_07FI007_0586.jpg",
    "FRAD084_07FI087_0267_01.jpg",
    "FRAD084_07FI074_0004.jpg",
    "FRAD084_07FI087_0186_01.jpg",
    "FRAD084_07FI007_0498.jpg",
    "FRAD084_07FI129_0001.jpg",
    "FRAD084_07FI007_0067.jpg",
    "FRAD084_07FI139_0094.jpg",
    "FRAD084_07FI031_0119.jpg",
    "FRAD084_07FI087_0081_01.jpg",
    "FRAD084_07FI072_0001.jpg",
    "FRAD084_07FI152_0004.jpg",
    "FRAD084_07FI139_0035.jpg",
    "FRAD084_07FI003_0027.jpg",
    "FRAD084_07FI007_0915.jpg",
    "FRAD084_07FI152_0071.jpg",
    "FRAD084_07FI007_0834.jpg",
    "FRAD084_07FI007_0714_01.jpg",
    "FRAD084_07FI007_0590.jpg",
    "FRAD084_07FI087_0251_01.jpg",
    "FRAD084_07FI007_1063_01.jpg",
    "FRAD084_07FI124_0003.jpg",
    "FRAD084_07FI007_0158.jpg",
    "FRAD084_07FI089_0029_01.jpg",
    "FRAD084_07FI132_0022_01.jpg",
    "FRAD084_07FI007_0819.jpg",
    "FRAD084_07FI087_0056.jpg",
    "FRAD084_07FI007_0375_01.jpg",
    "FRAD084_07FI042_0002.jpg",
    "FRAD084_07FI139_0059_01.jpg",
    "FRAD084_07FI152_0094.jpg",
    "FRAD084_07FI041_0033.jpg",
    "FRAD084_07FI007_0492_01.jpg",
    "FRAD084_07FI007_0670.jpg",
    "FRAD084_07FI050_0033.jpg",
    "FRAD084_07FI007_0965.jpg",
    "FRAD084_07FI031_0079_01.jpg",
    "FRAD084_07FI087_0298.jpg",
    "FRAD084_07FI139_0152.jpg",
    "FRAD084_07FI087_0300_01.jpg",
    "FRAD084_07FI007_0954.jpg",
    "FRAD084_07FI007_0950.jpg",
    "FRAD084_07FI019_0008_01.jpg",
    "FRAD084_07FI137_0030_01.jpg",
    "FRAD084_07FI087_0122.jpg",
    "FRAD084_07FI003_0073.jpg",
    "FRAD084_07FI152_0020.jpg",
    "FRAD084_07FI123_0042.jpg",
    "FRAD084_07FI081_0007_01.jpg",
    "FRAD084_07FI007_0414.jpg",
    "FRAD084_07FI017_0008.jpg",
    "FRAD084_07FI088_0023_01.jpg",
]

missing = []
for name in files:
    src = src_dir / name
    if not src.exists():
        missing.append(name)
        continue
    shutil.copy2(src, dst_dir / name)

print("copié:", len(files) - len(missing))
if missing:
    print("introuvables:", missing)


copié: 84


In [35]:
import json
from pathlib import Path

path = Path("./data/out_vaucluse/data_manuel.json")

data = json.loads(path.read_text(encoding="utf-8"))

for item in data:
    if "city" in item:
        item["city"] = ""
    if "monument_uniformise" in item:
        item["monument_uniformise"] = ""
    if "hamlet_uniformise" in item:
        item["hamlet_uniformise"] = ""
    item.pop("confidence", None)

path.write_text(json.dumps(data, ensure_ascii=False, indent=2), encoding="utf-8")
print("ok ->", path)


ok -> data/out_vaucluse/data_manuel.json


In [36]:
import json
from pathlib import Path

paths = [
    Path("data/out_vaucluse/data_manuel.json"),
    Path("data/out_vaucluse/transcriptions_coherentes_final.json"),
]

for path in paths:
    data = json.loads(path.read_text(encoding="utf-8"))
    data_sorted = sorted(data, key=lambda x: x.get("file_name", ""))
    path.write_text(json.dumps(data_sorted, ensure_ascii=False, indent=2), encoding="utf-8")
    print("ok ->", path)


ok -> data/out_vaucluse/data_manuel.json
ok -> data/out_vaucluse/transcriptions_coherentes_final.json


In [37]:
from pathlib import Path

src = Path("data/out_vaucluse/transcriptions_coherentes_final.json")
dst = Path("data/out_vaucluse/data_manuel.json")

dst.write_text(src.read_text(encoding="utf-8"), encoding="utf-8")
print("ok ->", dst)


ok -> data/out_vaucluse/data_manuel.json


In [40]:
import json
import unicodedata
import re
from pathlib import Path

path_manual = Path("data/out_vaucluse/data_manuel.json")
path_auto = Path("data/out_vaucluse/transcriptions_coherentes_final.json")

def normalize(s):
    s = (s or "").lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"\s+", " ", s).strip()
    return s

keys = ["city", "monument_uniformise", "hamlet_uniformise"]

manual = json.loads(path_manual.read_text(encoding="utf-8"))
auto = json.loads(path_auto.read_text(encoding="utf-8"))

manual_by_name = {d.get("file_name"): d for d in manual}
auto_by_name = {d.get("file_name"): d for d in auto}

stats = {k: {"total": 0, "errors": 0, "empty_vs_full": 0, "mismatch": 0} for k in keys}

# compte "Aucun lieu-dit"
target = normalize("Aucun lieu-dit")
count_manual = 0
count_auto = 0
count_both = 0

for fname, m in manual_by_name.items():
    a = auto_by_name.get(fname)
    if not a:
        continue

    for k in keys:
        m_val = normalize(m.get(k))
        a_val = normalize(a.get(k))
        stats[k]["total"] += 1

        if m_val != a_val:
            stats[k]["errors"] += 1
            if (not m_val and a_val) or (m_val and not a_val):
                stats[k]["empty_vs_full"] += 1
            else:
                stats[k]["mismatch"] += 1

    m_h = normalize(m.get("hamlet_uniformise"))
    a_h = normalize(a.get("hamlet_uniformise"))
    if m_h == target:
        count_manual += 1
    if a_h == target:
        count_auto += 1
    if m_h == target and a_h == target:
        count_both += 1

for k in keys:
    s = stats[k]
    print(f"{k}: total={s['total']} errors={s['errors']} empty_vs_full={s['empty_vs_full']} mismatch={s['mismatch']}")

print(f"hamlet_uniformise == 'Aucun lieu-dit' (manual): {count_manual}")
print(f"hamlet_uniformise == 'Aucun lieu-dit' (auto): {count_auto}")
print(f"hamlet_uniformise == 'Aucun lieu-dit' (both): {count_both}")


city: total=84 errors=4 empty_vs_full=0 mismatch=4
monument_uniformise: total=84 errors=16 empty_vs_full=8 mismatch=8
hamlet_uniformise: total=84 errors=7 empty_vs_full=2 mismatch=5
hamlet_uniformise == 'Aucun lieu-dit' (manual): 70
hamlet_uniformise == 'Aucun lieu-dit' (auto): 73
hamlet_uniformise == 'Aucun lieu-dit' (both): 70


In [41]:
import json
import unicodedata
import re
from pathlib import Path

path_manual = Path("data/out_vaucluse/data_manuel.json")
path_auto = Path("data/out_vaucluse/transcriptions_coherentes_final.json")

def normalize(s):
    s = (s or "").lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"\s+", " ", s).strip()
    return s

keys = ["city", "monument_uniformise", "hamlet_uniformise"]

manual = json.loads(path_manual.read_text(encoding="utf-8"))
auto = json.loads(path_auto.read_text(encoding="utf-8"))

manual_by_name = {d.get("file_name"): d for d in manual}
auto_by_name = {d.get("file_name"): d for d in auto}

stats = {
    k: {
        "total": 0,
        "errors": 0,
        "manual_empty_auto_full": 0,
        "manual_full_auto_empty": 0,
        "mismatch": 0,
    }
    for k in keys
}

# compte "Aucun lieu-dit"
target = normalize("Aucun lieu-dit")
count_manual = 0
count_auto = 0
count_both = 0

for fname, m in manual_by_name.items():
    a = auto_by_name.get(fname)
    if not a:
        continue

    for k in keys:
        m_val = normalize(m.get(k))
        a_val = normalize(a.get(k))
        stats[k]["total"] += 1

        if m_val != a_val:
            stats[k]["errors"] += 1
            if not m_val and a_val:
                stats[k]["manual_empty_auto_full"] += 1
            elif m_val and not a_val:
                stats[k]["manual_full_auto_empty"] += 1
            else:
                stats[k]["mismatch"] += 1

    m_h = normalize(m.get("hamlet_uniformise"))
    a_h = normalize(a.get("hamlet_uniformise"))
    if m_h == target:
        count_manual += 1
    if a_h == target:
        count_auto += 1
    if m_h == target and a_h == target:
        count_both += 1

for k in keys:
    s = stats[k]
    print(
        f"{k}: total={s['total']} errors={s['errors']} "
        f"manual_empty_auto_full={s['manual_empty_auto_full']} "
        f"manual_full_auto_empty={s['manual_full_auto_empty']} "
        f"mismatch={s['mismatch']}"
    )

print(f"hamlet_uniformise == 'Aucun lieu-dit' (manual): {count_manual}")
print(f"hamlet_uniformise == 'Aucun lieu-dit' (auto): {count_auto}")
print(f"hamlet_uniformise == 'Aucun lieu-dit' (both): {count_both}")


city: total=84 errors=4 manual_empty_auto_full=0 manual_full_auto_empty=0 mismatch=4
monument_uniformise: total=84 errors=16 manual_empty_auto_full=8 manual_full_auto_empty=0 mismatch=8
hamlet_uniformise: total=84 errors=7 manual_empty_auto_full=2 manual_full_auto_empty=0 mismatch=5
hamlet_uniformise == 'Aucun lieu-dit' (manual): 70
hamlet_uniformise == 'Aucun lieu-dit' (auto): 73
hamlet_uniformise == 'Aucun lieu-dit' (both): 70


In [3]:
import json
import unicodedata
import re
from pathlib import Path

path_manual = Path("data/out_vaucluse/data_manuel.json")
path_auto = Path("data/out_vaucluse/transcriptions_coherentes_final.json")

def normalize(s):
    s = (s or "").lower()
    s = unicodedata.normalize("NFKD", s)
    s = "".join(ch for ch in s if not unicodedata.combining(ch))
    s = re.sub(r"\s+", " ", s).strip()
    return s

keys = ["city", "monument_uniformise", "hamlet_uniformise"]

manual = json.loads(path_manual.read_text(encoding="utf-8"))
auto = json.loads(path_auto.read_text(encoding="utf-8"))

manual_by_name = {d.get("file_name"): d for d in manual}
auto_by_name = {d.get("file_name"): d for d in auto}

stats = {
    k: {
        "total": 0,
        "errors": 0,
        "manual_empty_auto_full": 0,
        "manual_full_auto_empty": 0,
        "mismatch": 0,
    }
    for k in keys
}

target_hamlet = normalize("Aucun lieu-dit")
target_monument = normalize("Aucun monument")
target_city = normalize("Aucune ville")

count_hamlet_manual = 0
count_hamlet_auto = 0
count_hamlet_both = 0

count_monu_manual = 0
count_monu_auto = 0
count_monu_both = 0

count_city_manual = 0
count_city_auto = 0
count_city_both = 0

for fname, m in manual_by_name.items():
    a = auto_by_name.get(fname)
    if not a:
        continue

    for k in keys:
        m_val = normalize(m.get(k))
        a_val = normalize(a.get(k))
        stats[k]["total"] += 1

        if m_val != a_val:
            stats[k]["errors"] += 1
            if not m_val and a_val:
                stats[k]["manual_empty_auto_full"] += 1
            elif m_val and not a_val:
                stats[k]["manual_full_auto_empty"] += 1
            else:
                stats[k]["mismatch"] += 1

    m_h = normalize(m.get("hamlet_uniformise"))
    a_h = normalize(a.get("hamlet_uniformise"))
    if m_h == target_hamlet:
        count_hamlet_manual += 1
    if a_h == target_hamlet:
        count_hamlet_auto += 1
    if m_h == target_hamlet and a_h == target_hamlet:
        count_hamlet_both += 1

    m_m = normalize(m.get("monument_uniformise"))
    a_m = normalize(a.get("monument_uniformise"))
    if m_m == target_monument:
        count_monu_manual += 1
    if a_m == target_monument:
        count_monu_auto += 1
    if m_m == target_monument and a_m == target_monument:
        count_monu_both += 1

    m_c = normalize(m.get("city"))
    a_c = normalize(a.get("city"))
    if m_c == target_city:
        count_city_manual += 1
    if a_c == target_city:
        count_city_auto += 1
    if m_c == target_city and a_c == target_city:
        count_city_both += 1

print('5 villes pour lesquelles on a pas le nom de la ville sur la carte')
for k in keys:
    s = stats[k]
    print(f"{k}: total={s['total']} errors={s['errors']} ")

print(" ")
print(f"hamlet_uniformise == 'Aucun lieu-dit' (manual): {count_hamlet_manual}")
print(f"hamlet_uniformise == 'Aucun lieu-dit' (auto): {count_hamlet_auto}")
print(f"hamlet_uniformise == 'Aucun lieu-dit' (both): {count_hamlet_both}")

print(" ")
print(f"monument_uniformise == 'Aucun monument' (manual): {count_monu_manual}")
print(f"monument_uniformise == 'Aucun monument' (auto): {count_monu_auto}")
print(f"monument_uniformise == 'Aucun monument' (both): {count_monu_both}")

print(" ")
print(f"city == 'Aucune ville' (manual): {count_city_manual}")
print(f"city == 'Aucune ville' (auto): {count_city_auto}")
print(f"city == 'Aucune ville' (both): {count_city_both}")


5 villes pour lesquelles on a pas le nom de la ville sur la carte
city: total=84 errors=9 
monument_uniformise: total=84 errors=17 
hamlet_uniformise: total=84 errors=7 
 
hamlet_uniformise == 'Aucun lieu-dit' (manual): 72
hamlet_uniformise == 'Aucun lieu-dit' (auto): 73
hamlet_uniformise == 'Aucun lieu-dit' (both): 70
 
monument_uniformise == 'Aucun monument' (manual): 30
monument_uniformise == 'Aucun monument' (auto): 21
monument_uniformise == 'Aucun monument' (both): 21
 
city == 'Aucune ville' (manual): 5
city == 'Aucune ville' (auto): 0
city == 'Aucune ville' (both): 0


In [55]:
!pip install matplotlib

In [52]:
!pip install pathlib

  Using cached pathlib-1.0.1-py3-none-any.whl.metadata (5.1 kB)
Using cached pathlib-1.0.1-py3-none-any.whl (14 kB)


In [4]:
import matplotlib.pyplot as plt
from pathlib import Path

out_dir = Path("data/out_vaucluse/plots")
out_dir.mkdir(parents=True, exist_ok=True)

# Données
total = 84
errors = {
    "city": 4,
    "monument_uniformise": 17,
    "hamlet_uniformise": 7,
}

hamlet_aucun = {
    "manual": 72,
    "auto": 73,
    "both": 70,
}

monu_aucun = {
    "manual": 30,
    "auto": 21,
    "both": 21,
}

city_aucune = {
    "manual": 5,  # <- remplace par tes valeurs
    "auto": 5,    # <- remplace par tes valeurs
    "both": 5,    # <- remplace par tes valeurs
}

# 1) Erreurs par champ
plt.figure(figsize=(6,4))
plt.bar(errors.keys(), errors.values(), color=["#4C78A8", "#F58518", "#54A24B"])
plt.title("Erreurs par champ (sur 84)")
plt.ylabel("Nombre d'erreurs")
plt.savefig(out_dir / "errors_par_champ.png", dpi=150, bbox_inches="tight")
plt.close()

# 2) Hamlet: Aucun lieu-dit
plt.figure(figsize=(6,4))
plt.bar(hamlet_aucun.keys(), hamlet_aucun.values(), color="#72B7B2")
plt.title("hamlet_uniformise = 'Aucun lieu-dit'")
plt.ylabel("Nombre")
plt.savefig(out_dir / "hamlet_aucun_lieu_dit.png", dpi=150, bbox_inches="tight")
plt.close()

# 3) Monument: Aucun monument
plt.figure(figsize=(6,4))
plt.bar(monu_aucun.keys(), monu_aucun.values(), color="#E45756")
plt.title("monument_uniformise = 'Aucun monument'")
plt.ylabel("Nombre")
plt.savefig(out_dir / "monument_aucun.png", dpi=150, bbox_inches="tight")
plt.close()

# 4) City: Aucune ville
plt.figure(figsize=(6,4))
plt.bar(city_aucune.keys(), city_aucune.values(), color="#9D755D")
plt.title("city = 'Aucune ville'")
plt.ylabel("Nombre")
plt.savefig(out_dir / "city_aucune_ville.png", dpi=150, bbox_inches="tight")
plt.close()

# 5) Taux d'erreurs (%)
plt.figure(figsize=(6,4))
rates = {k: (v / total) * 100 for k, v in errors.items()}
plt.bar(rates.keys(), rates.values(), color="#B279A2")
plt.title("Taux d'erreur par champ (%)")
plt.ylabel("Pourcentage")
plt.savefig(out_dir / "taux_erreurs.png", dpi=150, bbox_inches="tight")
plt.close()

print("ok ->", out_dir)


ok -> data/out_vaucluse/plots
